In [ ]:
import os 
nameof_killfile="ray::IDLE"

if os.path.exists(nameof_killfile): 
    os.remove(nameof_killfile) 
else:
    killit=open(nameof_killfile,"w") 
    killit.close()

In [ ]:
import os
import subprocess

subprocess = subprocess.Popen(['ps', '-A'], stdout=subprocess.PIPE)
output, error = subprocess.communicate()

target_process = "ray::IDLE"
for line in output.splitlines():
    if target_process in str(line):
        pid = int(line.split(None, 1)[0])
        os.kill(pid, 9)

In [1]:
from arcgis.gis import GIS
my_gis = GIS()
m = my_gis.map()
m

MapView(layout=Layout(height='400px', width='100%'))

In [12]:
import os, ray

ray.init()

@ray.remote(num_gpus=1)
def use_gpu():
    print("ray.get_gpu_ids(): {}".format(ray.get_gpu_ids()))
    print("CUDA_VISIBLE_DEVICES: {}".format(os.environ["CUDA_VISIBLE_DEVICES"]))

2020-06-15 03:11:37,463	INFO resource_spec.py:212 -- Starting Ray with 152.34 GiB memory available for workers and up to 69.29 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-06-15 03:11:37,828	INFO services.py:1170 -- View the Ray dashboard at localhost:8267


In [13]:
use_gpu.remote()

ObjectID(45b95b1c8bd3a9c4ffffffff010000c801000000)

(pid=95580) ray.get_gpu_ids(): [0]
(pid=95580) CUDA_VISIBLE_DEVICES: 0


In [14]:
os.environ["CUDA_VISIBLE_DEVICES"]="512"

In [15]:
use_gpu.remote()

ObjectID(ef0a6c221819881cffffffff010000c801000000)

(pid=95580) ray.get_gpu_ids(): [0]
(pid=95580) CUDA_VISIBLE_DEVICES: 0


In [16]:
import tensorflow
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1885564196234881240
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 12225244859284870458
physical_device_desc: "device: XLA_CPU device"
]


In [15]:
import torch
print("Cuda available: ", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name())

Cuda available:  False


RuntimeError: cuda runtime error (38) : no CUDA-capable device is detected at /pytorch/aten/src/THC/THCGeneral.cpp:50

In [16]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2158894899230587998
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 14079766441921586453
physical_device_desc: "device: XLA_CPU device"
]


In [17]:
from keras import backend as K
print(K.tensorflow_backend._get_available_gpus())

AttributeError: module 'tensorflow._api.v2.config' has no attribute 'experimental_list_devices'

In [2]:
import numpy as np
import numba

@numba.stencil
def _smooth(x):
    return (x[-1, -1] + x[-1, 0] + x[-1, 1] +
            x[ 0, -1] + x[ 0, 0] + x[ 0, 1] +
            x[ 1, -1] + x[ 1, 0] + x[ 1, 1]) // 9

@numba.njit
def smooth_cpu(x):
    return _smooth(x)

In [3]:
from numba import cuda

@cuda.jit
def smooth_gpu(x, out):
    i, j = cuda.grid(2)
    n, m = x.shape
    if 1 <= i < n - 1 and 1 <= j < m - 1:
        out[i, j] = (x[i - 1, j - 1] + x[i - 1, j] + x[i - 1, j + 1] +
                     x[i    , j - 1] + x[i    , j] + x[i    , j + 1] +
                     x[i + 1, j - 1] + x[i + 1, j] + x[i + 1, j + 1]) // 9

In [4]:
x_cpu = np.ones((10000, 10000), dtype='int8')

%timeit smooth_cpu(x_cpu)

426 ms ± 2.39 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
import cupy,math

x_gpu = cupy.ones((10000, 10000), dtype='int8')
out_gpu = cupy.zeros((10000, 10000), dtype='int8')

# I copied the four lines below from the Numba docs
threadsperblock = (16, 16)
blockspergrid_x = math.ceil(x_gpu.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(x_gpu.shape[1] / threadsperblock[1])
blockspergrid = (blockspergrid_x, blockspergrid_y)

%timeit smooth_gpu[blockspergrid, threadsperblock](x_gpu, out_gpu)

ImportError: CuPy is not correctly installed.

If you are using wheel distribution (cupy-cudaXX), make sure that the version of CuPy you installed matches with the version of CUDA on your host.
Also, confirm that only one CuPy package is installed:
  $ pip freeze

If you are building CuPy from source, please check your environment, uninstall CuPy and reinstall it with:
  $ pip install cupy --no-cache-dir -vvvv

Check the Installation Guide for details:
  https://docs-cupy.chainer.org/en/latest/install.html

original error: libcublas.so.10: cannot open shared object file: No such file or directory

In [1]:
import cupy
print(cupy.show_config())


ImportError: CuPy is not correctly installed.

If you are using wheel distribution (cupy-cudaXX), make sure that the version of CuPy you installed matches with the version of CUDA on your host.
Also, confirm that only one CuPy package is installed:
  $ pip freeze

If you are building CuPy from source, please check your environment, uninstall CuPy and reinstall it with:
  $ pip install cupy --no-cache-dir -vvvv

Check the Installation Guide for details:
  https://docs-cupy.chainer.org/en/latest/install.html

original error: libcublas.so.10: cannot open shared object file: No such file or directory